<a href="https://colab.research.google.com/github/buketugurlu/LangTutorAI/blob/main/dilogrenmeyardimcisi5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Gerekli Kütüphaneleri Yükleme
!pip install transformers datasets torch scikit-learn

import os
import torch
from datasets import load_dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# 1. Cihaz Seçimi
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan cihaz: {device}")

# 2. Model ve Tokenizer Yükleme
model_name = "modfiededition/t5-base-fine-tuned-on-jfleg"
model = T5ForConditionalGeneration.from_pretrained(model_name, from_tf=True).to(device) # Add from_tf=True
tokenizer = T5Tokenizer.from_pretrained(model_name)

# 3. Veri Setini Yükleme ve İşleme
dataset = load_dataset("jfleg", split="validation")  # 'validation' split
data = [{"input": f"grammar: {x['sentence']}", "target": x["corrections"][0]} for x in dataset]

# Eğitim ve doğrulama setlerini ayır
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

class GrammarCorrectionDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        source = self.data[idx]["input"]
        target = self.data[idx]["target"]

        source_enc = self.tokenizer(
            source, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        target_enc = self.tokenizer(
            target, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )

        return {
            "input_ids": source_enc["input_ids"].squeeze(0),
            "attention_mask": source_enc["attention_mask"].squeeze(0),
            "labels": target_enc["input_ids"].squeeze(0)
        }

train_dataset = GrammarCorrectionDataset(train_data, tokenizer)
val_dataset = GrammarCorrectionDataset(val_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

# 4. Eğitim Ayarları
optimizer = AdamW(model.parameters(), lr=5e-5)
epochs = 3

def train_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(data_loader)

def eval_epoch(model, data_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            total_loss += loss.item()
    return total_loss / len(data_loader)

# 5. Eğitim Döngüsü
for epoch in range(epochs):
    train_loss = train_epoch(model, train_loader, optimizer, device)
    val_loss = eval_epoch(model, val_loader, device)
    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Validation Loss: {val_loss:.4f}")

# 6. Model Kaydetme
model_dir = "t5_fine_tuned_grammar"
os.makedirs(model_dir, exist_ok=True)
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

# 7. Model Testi
def test_sentence(sentence):
    input_text = f"grammar: {sentence}"
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).input_ids.to(device)
    outputs = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

example_sentence = "She go to school every day and like to learn new things."
corrected_sentence = test_sentence(example_sentence)

print(f"Orijinal Cümle: {example_sentence}")
print(f"Düzeltilmiş Cümle: {corrected_sentence}")


Kullanılan cihaz: cpu


tf_model.h5:   0%|          | 0.00/892M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/141k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1/3
Train Loss: 3.8100
Validation Loss: 0.1954
Epoch 2/3
Train Loss: 0.2561
Validation Loss: 0.1657
Epoch 3/3
Train Loss: 0.1682
Validation Loss: 0.1290
Orijinal Cümle: She go to school every day and like to learn new things.
Düzeltilmiş Cümle: She goes to school every day and likes to learn new things.


In [20]:
# 7. Model Testi
def test_sentence(sentence):
    input_text = f"grammar: {sentence}"
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).input_ids.to(device)
    outputs = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

example_sentence = "Yesterday, he eat an apple.."
corrected_sentence = test_sentence(example_sentence)

print(f"Orijinal Cümle: {example_sentence}")
print(f"Düzeltilmiş Cümle: {corrected_sentence}")

Orijinal Cümle: Yesterday, he eat an apple..
Düzeltilmiş Cümle: Yesterday, he ate an apple.
